In [31]:
import requests
import nltk
from nltk.corpus import stopwords
from string import punctuation
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from bs4 import BeautifulSoup
from nltk.sentiment import vader
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [32]:
# Step 2

# Grab the UTF-8 plain text for Macbeth from the Gutenberg Project
plain_text = requests.get("https://www.gutenberg.org/ebooks/2264.txt.utf-8").text

# Tokenize that text into words
# Lowercase all the words
words = word_tokenize(plain_text.lower())

# Create a new list with no punctuation or stopwords
stop_words = stopwords.words("english") + list(punctuation)
filtered = [word for word in words if word not in stop_words]
        
# Find the frequency of all words and pick the top 20
freq = FreqDist(filtered)
for i in sorted(freq, key=freq.get, reverse=True)[:20]:
    print(i,freq[i])

'd 192
's 151
macb 137
haue 122
thou 87
enter 81
shall 68
macbeth 67
thee 61
vpon 59
macd 58
yet 57
come 56
thy 56
vs 56
time 54
king 54
hath 52
may 51
good 49


# Meaning of most frequent words

These words definitely do not give a good sense of what the play is about. It might tell someone that it involves a king but  
it seems as though the tokenization and removal of stopwords still left in a lot of garble. This may be due to the fact that  
Shakespeare wrote in Old English and it is hard for nltk to parse it properly.

In [36]:
# Step 3:

# Yelp reviews for Lamonica's NY Pizza
reviews = [
"When it comes to the Pizza Wars in Westwood, Lamonica's gets my vote. Their pizza is PERFECT. I've had both the cheese and sausage and I could eat both of these slices multiple times a week. Prices are reasonable and they had a fantastic deal on their anniversary this year. I'm a UCLA student and am always excited when I attend events where Lamonica's pizza is served! There is limited seating, so if you can't find a place to sit and eat, I would recommend parking in the Broxton parking lot to pick up your pizza and then heading home to enjoy it!",
"If you're looking for a pizza place in westwood, I think this is THE PLACE to go. Their pizza is made on the spot, and is sooo delish! We ordered a large 18\" cheese pizza for lunch, and it was enough to fill 3 people even after a 6 hour drive from NorCal. Our order took about 10 minutes to be ready. The cheesiness, the crust, the flavor, everything was basically perfect. The pizza wasn't too greasy, the crust was super soft and chewy, and the pizza wasn't paper thin. Although I've never had NY pizza from NY, if this is what it's like, then I need to book myself a flight to NY ASAP. In terms of local UCLA pizza wars, this place is a little pricey, but if you're looking for premium pizza and the ultimate winner of pizza, this is the place to go.",
"Came here on the day listed in the picture after hearing about cheap food and shared an entire pie with a friend who I went with.  TLDR:  definitely worth it for the $7 we paid but I don't think I'd come here at the regular price. Pros: Pizza was definitely New York style Could see the new dough proofing so it's clear that they make their own dough Adequate amount of sauce Some burnt bits on the bottom. Yum Close to UCLA campus Cons: Normal price for just a cheese pizza is like $20?? Toppings jack up the price like crazy Overall, definitely good New York style pizza but not pizza from New York kinda good.  Worth a visit if you're missing it but not if you're looking for a cheap eat.",
"Very expensive and not as good as real new york pizza. We got the lunch special so it was cheaper but just Nothing Special.",
"Tried the mixed meat and pepperoni pizza, both were good. Like all pizza, a little greasy. Had some flop, but not a lot. Situated near UCLA in Westwood Villiage.",
"Pizza was good. I give high marks for The za. However, an employee asked uniformed police officers for their vax cards. They laughed and walked out, bravo. I cannot support a place that pushes the unconstitutional bs of vax cards.",
"Only the best. Been going to Lamonica's NY Pizza since 1992 and I consider it the best. They actually fly-in the dough from New York. Since it's by the slice, it's always ready. My favorite is sausage because it tastes amazing. Pepperoni is also good.",
"I have been ordering pizzas from LaMonicas for nearly 15 years. This was the worst experience I have had by far. The manager was extremely rude, would not take care of my request to remake the pizza I ordered. The pizza was over cooked and had extremely thin and flat crust; not a nice sizable crust like they usually make them. He kept repeating that this is the way their pizzas always are (they were not) and when I told him that the customer is right, he told me not at their establishment! He ignored me and was rude to no end. I was calm and did not deserve that treatment and honestly I am still shocked at how rude he was being. Do not give them your business. There are many other places with good pizza that would treat their customers with care.",
"I've been coming to Lamonicas since it opened and it was consistently good. Maybe there is a new owner or manager because the quality last night was disappointing.  The cheese slice I ordered was stiff and had burn marks on the bottom.  It's the same sensation as eating cardboard.  When i asked for a different slice that didn't crunch when you eat it the cashier told me the pizzas are cooked that way now and another slice wouldn't be any different.  I also ordered garlic knots that were burned on the top and too tough inside to eat.  They went to the trash. I hope this was an \"off night\". It used to be the best pizza in Westwood.",
"If you are looking for a classic NY pizza slice, I would recommend Lamonica's. They've been around for quite some time and still live up to their name. Their pizzas are well proportioned and not too heavy. This place is great for a quick, delicious slice.",
"Got the veggie and cheese pizza here for lunch and I was very disappointed. For the veggie pizza everything completely fell off the pizza the veggies and cheese which was disappointing. It was also very dry and the crust was tuff. The only positive comment I would have to say is that the tomatoes sauce did have flavor that elevated the pizza and the portion sizes were big. If it weren't for the tomatoes sauce I don't think I would have eaten it. I do want to try the Sicilian pizza however and see if it's any better",
"Extraordinary pizza here. Tastes just like New York City. I think this is the best NY style pizza in LA. Seating is very limited and so is parking since it's located in the middle of Westwood, but the pizza comes out fast.",
"Out of all the places that claim they have New York pizza, this establishment is the truth hands down. Thanks to Lorena I am a frequent customer since she told me about this location. The pizza here is exactly the same pizza I get from New York & the staff here are the real MVP.",
"Man why haven't I tried this pizza place before? I have lived in California my entire life and have never been here, if you live in area or traveling back from airport stop here you won't regret it.",
"For weeks, Lamonica's had flyers up about anniversary specials today on October 19, including 3 topping pizzas for $10. I went in today, October 19, to find they were going to charge me $16 and change for a 3 topping pizza with meatball, olive, and mushroom. They added a sheet that said that any non-pepperoni, mushroom, or sausage topping would be around an additional $3. Are customers supposed to visit the business every single day to find out about these additional costs being added to the alleged special just because it was nowhere in the flyer that had been up for weeks. There was no reason to get the 3 topping pizza I ordered because of this. This is deceptive and not a good way to do business."
]
ratings = [5, 4, 4, 3, 4, 2, 5, 1, 2, 5, 3, 5, 5, 5, 1]

# Use Vader to find polarity of each review
sia = vader.SentimentIntensityAnalyzer()
polarity = [sia.polarity_scores(review) for review in reviews]
for rating, polar in zip(ratings, polarity):
    print("Yelp rating: " + str(rating) + " vs Vader: " + str(polar['compound']))

Yelp rating: 5 vs Vader: 0.9046
Yelp rating: 4 vs Vader: 0.8883
Yelp rating: 4 vs Vader: 0.9817
Yelp rating: 3 vs Vader: -0.4098
Yelp rating: 4 vs Vader: 0.25
Yelp rating: 2 vs Vader: 0.5635
Yelp rating: 5 vs Vader: 0.9666
Yelp rating: 1 vs Vader: -0.9356
Yelp rating: 2 vs Vader: 0.7964
Yelp rating: 5 vs Vader: 0.9081
Yelp rating: 3 vs Vader: 0.0534
Yelp rating: 5 vs Vader: 0.4124
Yelp rating: 5 vs Vader: 0.6369
Yelp rating: 5 vs Vader: 0.0341
Yelp rating: 1 vs Vader: -0.2278


In [37]:
# Step 4:

# Reviews for Kung Fu Panda and Good Will Hunting from Rotten Tomatoes
movie_reviews = [
"There's something about the look of Kung Fu Panda that is so novel and pleasing. Perhaps it's just that we haven't seen this vintage, Far East world in a computer-animated movie before.",
"As the credits roll, the film's many flaws come sharply into focus. But while it lasts, Kung-Fu Panda is something of a guilty treat.",
"The animation work is dazzling; it's lovingly detailed without being overdone (particularly the opening sequence, which is hand-drawn and looks like prints struck from ancient woodblocks).",
"It's not the first or the last film to sermonise the idea of \"never giving up,\" but there is a certain charm in seeing a collector of action figures, an indolent slug, turn into an animal-bashing Bruce Lee.",
"Manages to entertain and dazzle, and parents won't beat themselves up for sitting through it, although what it teaches could fit on a fortune cookie.",
"The headline story from this slice of honestly earned sentiment is Matt Damon, who delivers the year's No. 1 breakthrough performance directly atop his agreeable high-profile turn in John Grisham's The Rainmaker.",
"It's a movie that captures (in a way that perhaps an older screenwriter could not convey) the limitless possibilities of youth, a movie in which a character can tell another, without cynicism, \"You could do anything you want. You are bound by nothing.\"",
"The best thing about Good Will Hunting is not in its well-crafted, psychological symmetries but in the just-plain messiness of its humanity. It's rowdy, it's funny, it's heartbreaking -- it rings of life.",
"Towering performance by Matt Damon as a troubled working class who needs to address his creative genius elevates this drama way above its therapeutic approach, resulting in a zeitgeist film that may touch chord with young viewers the way The Graduate did",
"It's writer-stars Matt Damon & Ben Affleck's childhood friendship-chemistry and the hilarious bickering between the real-life Affleck brothers that generates the magic in \"Good Will Hunting.\" Not to mention a magical performance by Robin Williams."
]

extrastop = ['``',"''","'re","'s","'re",'``',"''","'ll","--","\'\'","...",
             "n\'t",'one','would','use','subject','from',"\'m","\'ve"]

myStopWords = list(punctuation) + stopwords.words('english') + extrastop

listOfMovieWords = []
for i in movie_reviews:
    listOfMovieWords.append([w for w in word_tokenize(i.lower()) if w not in myStopWords])
    
from nltk.stem.porter import PorterStemmer

p_stemmer = PorterStemmer()

listOfStemmedWords = []
for i in listOfMovieWords:
    listOfStemmedWords.append([p_stemmer.stem(w) for w in i])
    
!pip install gensim

from gensim import corpora, models
import gensim

dictionary = corpora.Dictionary(listOfStemmedWords)

print(dictionary.token2id)

     |████████████████████████████████| 24.0 MB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 2.1 MB/s  eta 0:00:01
{'computer-anim': 0, 'east': 1, 'far': 2, 'fu': 3, 'kung': 4, 'look': 5, 'movi': 6, 'novel': 7, 'panda': 8, 'perhap': 9, 'pleas': 10, 'seen': 11, 'someth': 12, 'vintag': 13, 'world': 14, 'come': 15, 'credit': 16, 'film': 17, 'flaw': 18, 'focu': 19, 'guilti': 20, 'kung-fu': 21, 'last': 22, 'mani': 23, 'roll': 24, 'sharpli': 25, 'treat': 26, 'ancient': 27, 'anim': 28, 'dazzl': 29, 'detail': 30, 'hand-drawn': 31, 'like': 32, 'lovingli': 33, 'open': 34, 'overdon': 35, 'particularli': 36, 'print': 37, 'sequenc': 38, 'struck': 39, 'without': 40, 'woodblock': 41, 'work': 42, 'action': 43, 'animal-bash': 44, 'bruce': 45, 'certain': 46, 'charm': 47, 'collector': 48, 'figur': 49, 'first': 50, 'give': 51, 'idea': 52, 'indol': 53, 'lee': 54, 'never': 55, 'see': 56, 'sermonis': 57, 'slug': 58, 'turn': 59, 'although': 60, 'beat': 61, 'cooki': 62, 'could': 63, 'ent

In [38]:
corpus = [dictionary.doc2bow(text) for text in listOfStemmedWords]

IndexError: list index out of range

In [40]:
print(corpus[2])

[(5, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)]


In [41]:
print(dictionary.token2id['parent'])

68


In [42]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=2, 
                                           id2word = dictionary, 
                                           passes=20)

In [44]:
for i in ldamodel.print_topics(num_topics=2, num_words=20):
    print(i)

(0, '0.017*"movi" + 0.017*"could" + 0.017*"without" + 0.010*"perhap" + 0.010*"look" + 0.010*"panda" + 0.010*"someth" + 0.010*"dazzl" + 0.010*"hunt" + 0.010*"good" + 0.010*"last" + 0.010*"work" + 0.010*"convey" + 0.010*"screenwrit" + 0.010*"noth" + 0.010*"anoth" + 0.010*"want" + 0.010*"captur" + 0.010*"anyth" + 0.010*"charact"')
(1, '0.018*"damon" + 0.018*"perform" + 0.018*"matt" + 0.013*"film" + 0.013*"way" + 0.013*"magic" + 0.013*"affleck" + 0.013*"turn" + 0.008*"therapeut" + 0.008*"young" + 0.008*"zeitgeist" + 0.008*"need" + 0.008*"geniu" + 0.008*"tower" + 0.008*"drama" + 0.008*"chord" + 0.008*"graduat" + 0.008*"may" + 0.008*"elev" + 0.008*"result"')


# LDA Words Split

These words absolutely represent the split that I had for reviews of Kung Fu Panda vs Good Will Hunting. The fact that topic 1 contains the words panda, capture, dazzle, and screenwriting kind of hints to the fact that critics raved about how Kung Fu Panda had done extremely well when it came to bringing cartoon characters and animals such as a panda to life. Topic 2 also had mentioned many things about actors Ben Affleck and Matt Damon as well as the previous movie that Matt Damon had which was The Graduate.